# 🏷️ BNNR — Multi-Label Classification Demo

**Bulletproof Neural Network Recipe** — Automated Augmentation Search for PyTorch.

This notebook demonstrates **multi-label classification** with BNNR — where each image can have **multiple labels simultaneously** (multi-hot encoding).

You'll see:

- **Multi-hot labels** — each image has 1-4 active labels out of 6 possible
- `SimpleTorchAdapter(multilabel=True)` — auto-selects BCEWithLogitsLoss + sigmoid thresholding
- `BNNRConfig(task="multilabel")` — auto-selects F1 (samples) as the primary metric
- All 8 built-in BNNR augmentations (7 unique + BasicAug) + **ICD/AICD** (XAI-driven)
- OptiCAM saliency maps showing which regions drive each label
- Iterative augmentation selection optimising F1 score

**Single-label vs Multi-label:**

| Aspect | Single-label | Multi-label |
|:---|:---|:---|
| Labels | One-hot `[0,0,1,0,0]` | Multi-hot `[0,1,1,0,1]` |
| Loss | CrossEntropyLoss | BCEWithLogitsLoss |
| Predictions | argmax → class index | sigmoid > threshold → binary vector |
| Primary metric | Accuracy | F1 (samples) |

⏱ **Runtime:** ~3 min on CPU, ~1 min on GPU.

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/mateuszwalo/BNNR/blob/main/examples/multilabel/bnnr_multilabel_demo.ipynb)


## 1. Installation

In [ ]:
%pip install -q "bnnr[dashboard]" matplotlib

## 2. Imports

In [ ]:
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader, Dataset, TensorDataset

from bnnr import (
    BNNRConfig,
    BNNRTrainer,
    SimpleTorchAdapter,
    AugmentationRegistry,
)
from bnnr.icd import ICD, AICD
from bnnr.xai_cache import XAICache
from bnnr.augmentations import BaseAugmentation

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
SEED = 42
print(f"Using device: {DEVICE}")

## 3. Synthetic Multi-Label Dataset

We create a synthetic dataset where each image can exhibit **multiple visual attributes**:

| Label | Visual Feature |
|:------|:---------------|
| `red_tint` | Warm red colour shift |
| `blue_tint` | Cool blue colour shift |
| `high_contrast` | Sharp horizontal edge |
| `striped` | Horizontal stripe pattern |
| `spotted` | Circular bright spots |
| `bright` | Overall brightness boost |

Each image gets 1–4 random attributes → multi-hot label vector.

The `IndexedDataset` wrapper adds a sample index — required for BNNR's XAI caching (ICD/AICD).

In [ ]:
N_LABELS = 6
IMG_SIZE = 48
LABEL_NAMES = ["red_tint", "blue_tint", "high_contrast", "striped", "spotted", "bright"]


class IndexedDataset(Dataset):
    """Wraps (image, label) → (image, label, index) for ICD/AICD cache."""
    def __init__(self, ds):
        self.ds = ds
    def __len__(self):
        return len(self.ds)
    def __getitem__(self, idx):
        img, lbl = self.ds[idx]
        return img, lbl, idx


def make_multilabel_dataset(n, seed=SEED):
    """Create synthetic multi-label RGB images with random attribute combinations."""
    rng = np.random.RandomState(seed)
    images, labels = [], []

    for _ in range(n):
        img = rng.rand(3, IMG_SIZE, IMG_SIZE).astype(np.float32) * 0.15
        n_active = rng.randint(1, 5)
        active = rng.choice(N_LABELS, size=n_active, replace=False)
        label = np.zeros(N_LABELS, dtype=np.float32)
        label[active] = 1.0

        for a in active:
            if a == 0:    # red_tint
                img[0] += rng.uniform(0.3, 0.6)
            elif a == 1:  # blue_tint
                img[2] += rng.uniform(0.3, 0.6)
            elif a == 2:  # high_contrast
                row = rng.randint(IMG_SIZE // 4, 3 * IMG_SIZE // 4)
                img[:, :row, :] += 0.3
                img[:, row:, :] -= 0.1
            elif a == 3:  # striped
                for r in range(0, IMG_SIZE, 4):
                    img[:, r:r+2, :] += 0.25
            elif a == 4:  # spotted
                for _ in range(rng.randint(3, 6)):
                    cy, cx = rng.randint(4, IMG_SIZE - 4, size=2)
                    radius = rng.randint(2, 5)
                    yy, xx = np.ogrid[-cy:IMG_SIZE-cy, -cx:IMG_SIZE-cx]
                    mask = (xx**2 + yy**2) <= radius**2
                    img[:, mask] += 0.35
            elif a == 5:  # bright
                img += rng.uniform(0.2, 0.4)

        img += rng.randn(3, IMG_SIZE, IMG_SIZE).astype(np.float32) * 0.05
        images.append(torch.from_numpy(np.clip(img, 0, 1)))
        labels.append(torch.from_numpy(label))

    return TensorDataset(torch.stack(images), torch.stack(labels))


# Create datasets
N_TRAIN, N_VAL = 300, 150
train_ds = make_multilabel_dataset(N_TRAIN, seed=SEED)
val_ds = make_multilabel_dataset(N_VAL, seed=SEED + 1000)

train_loader = DataLoader(IndexedDataset(train_ds), batch_size=32, shuffle=True)
val_loader = DataLoader(IndexedDataset(val_ds), batch_size=32)

# Label statistics
all_labels = train_ds.tensors[1].numpy()
print(f"Train: {N_TRAIN}  Val: {N_VAL}  Labels: {N_LABELS}")
print(f"Avg labels/image: {all_labels.sum(axis=1).mean():.1f}")
print(f"Per-label counts: {dict(zip(LABEL_NAMES, all_labels.sum(axis=0).astype(int)))}")

### Visualise some samples

Let's look at a few images and their multi-hot labels to understand the dataset.

In [ ]:
fig, axes = plt.subplots(2, 5, figsize=(15, 6))
for i, ax in enumerate(axes.flat):
    img = train_ds[i][0].permute(1, 2, 0).numpy()
    lbl = train_ds[i][1].numpy()
    active = [LABEL_NAMES[j] for j in range(N_LABELS) if lbl[j] > 0.5]
    ax.imshow(img)
    ax.set_title("\n".join(active), fontsize=8)
    ax.axis("off")
fig.suptitle("Sample Images with Multi-Hot Labels", fontsize=14, y=1.02)
plt.tight_layout()
plt.show()

## 4. Model

A compact CNN for multi-label classification. Key differences from single-label:

- **Output size** = number of labels (not classes) — 6 raw logits
- **No softmax/sigmoid** at the end — `BCEWithLogitsLoss` handles it internally
- `target_layer` property points to the last conv for OptiCAM saliency maps

In [ ]:
class MultiLabelCNN(nn.Module):
    def __init__(self, n_labels=N_LABELS):
        super().__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 32, 3, padding=1), nn.BatchNorm2d(32), nn.ReLU(True),
            nn.MaxPool2d(2),
            nn.Conv2d(32, 64, 3, padding=1), nn.BatchNorm2d(64), nn.ReLU(True),
            nn.MaxPool2d(2),
            nn.Conv2d(64, 128, 3, padding=1),  # target layer for OptiCAM
            nn.BatchNorm2d(128), nn.ReLU(True),
            nn.AdaptiveAvgPool2d((1, 1)),
        )
        self.classifier = nn.Sequential(
            nn.Flatten(), nn.Dropout(0.3),
            nn.Linear(128, n_labels),  # raw logits, no sigmoid!
        )

    def forward(self, x):
        return self.classifier(self.features(x))

    @property
    def target_layer(self):
        return self.features[8]  # Last Conv2d


model = MultiLabelCNN()
print(f"Parameters: {sum(p.numel() for p in model.parameters()):,}")
print(f"Output: {N_LABELS} logits (one per label)")

## 5. Adapter & Config — the two key changes

For multi-label classification, you only need **two changes** compared to single-label:

### `SimpleTorchAdapter(multilabel=True)`
This tells the adapter to:
- Cast labels to `float()` (required by BCEWithLogitsLoss)
- Use `sigmoid(logits) >= 0.5` for predictions (instead of `argmax`)
- Compute `f1_samples`, `f1_macro`, `accuracy` as eval metrics

### `BNNRConfig(task="multilabel")`
This tells the trainer to:
- Auto-set `selection_metric = "f1_samples"` (best metric for multi-label)
- Auto-set `metrics = ["f1_samples", "f1_macro", "accuracy", "loss"]`
- Use per-label F1 analysis instead of confusion matrix
- Handle multi-hot labels in data profiling, XAI, and probe selection

In [ ]:
# ── Adapter ──────────────────────────────────────────────────────────
adapter = SimpleTorchAdapter(
    model=model,
    criterion=nn.BCEWithLogitsLoss(),         # multi-label loss
    optimizer=torch.optim.AdamW(model.parameters(), lr=1e-3, weight_decay=1e-4),
    target_layers=[model.target_layer],
    device=DEVICE,
    multilabel=True,                          # ← THE KEY FLAG
    multilabel_threshold=0.5,                 # prediction threshold
)

print(f"Loss:    {adapter.criterion}")
print(f"Metrics: {adapter.eval_metrics}")

# ── Config ───────────────────────────────────────────────────────────
from pathlib import Path

output_dir = Path("multilabel_notebook_output")

config = BNNRConfig(
    task="multilabel",                        # ← THE KEY FLAG
    m_epochs=3,                               # epochs per branch
    max_iterations=3,                         # 3 decision rounds
    device=DEVICE,
    xai_enabled=True,
    xai_method="opticam",
    xai_samples=2,
    report_dir=output_dir / "reports",
    checkpoint_dir=output_dir / "checkpoints",
    xai_cache_dir=output_dir / "xai_cache",
    xai_cache_samples=N_TRAIN,
    save_checkpoints=True,
    verbose=True,
    event_log_enabled=True,
    report_preview_size=128,
    report_xai_size=128,
    report_probe_images_per_class=2,
    report_probe_max_classes=N_LABELS,
    candidate_pruning_enabled=True,
    candidate_pruning_warmup_epochs=1,
    candidate_pruning_relative_threshold=0.5,
    seed=SEED,
)

print(f"\ntask:             {config.task}")
print(f"selection_metric: {config.selection_metric}  (auto-set!)")
print(f"metrics:          {config.metrics}  (auto-set!)")

## 6. XAI Cache & Augmentations

**XAI cache** precomputes saliency maps for ICD/AICD, making training much faster.

We register all 8 built-in BNNR augmentations (7 unique + BasicAug) + ICD + AICD. Multi-hot labels are handled transparently — ICD/AICD use the **dominant class** (highest logit) for saliency computation.

In [ ]:
# ── XAI Cache ────────────────────────────────────────────────────────
xai_cache = XAICache(output_dir / "xai_cache")
written = xai_cache.precompute_cache(
    model=model, train_loader=train_loader,
    target_layers=[model.target_layer],
    n_samples=N_TRAIN, method="opticam",
)
print(f"Cached {written} saliency maps\n")


# ── Augmentations ────────────────────────────────────────────────────
def _named(name, display_name, **kw):
    aug = AugmentationRegistry.create(name, **kw)
    aug.name = display_name
    return aug

augmentations = [
    _named("basic_augmentation", "basic_augmentation", probability=0.5, random_state=SEED),
    _named("church_noise",       "church_noise",       probability=0.5, random_state=SEED+1),
    _named("dif_presets",        "dif_presets",        probability=0.5, random_state=SEED+2),
    _named("drust",              "drust",              probability=0.5, random_state=SEED+3),
    _named("luxfer_glass",       "luxfer_glass",       probability=0.5, random_state=SEED+4),
    _named("procam",             "procam",             probability=0.5, random_state=SEED+5),
    _named("smugs",              "smugs",              probability=0.5, random_state=SEED+6),
    _named("tea_stains",         "tea_stains",         probability=0.5, random_state=SEED+7),
]

# ICD & AICD — work with multi-hot labels out of the box
icd = ICD(model=model, target_layers=[model.target_layer],
          cache=xai_cache, probability=0.5, random_state=SEED+10)
icd.name = "icd"
augmentations.append(icd)

aicd = AICD(model=model, target_layers=[model.target_layer],
            cache=xai_cache, probability=0.5, random_state=SEED+11)
aicd.name = "aicd"
augmentations.append(aicd)

print(f"{len(augmentations)} augmentation candidates:")
for i, a in enumerate(augmentations, 1):
    print(f"  {i:2d}. {a.name}")

## 7. Dashboard (Live + Colab + Mobile)

Start dashboard before training so you can monitor branch decisions and multilabel metrics in real time.

Tracking modes:
- **Local machine**: open local URL (terminal workflows provide QR for phone access).
- **Google Colab**: iframe appears directly in notebook output.
- **Optional external/mobile URL on Colab**: configure `NGROK_AUTHTOKEN` and use `pyngrok`.

Minimum validation during run:
- KPI cards update each epoch,
- sample/XAI section renders without errors,
- `events.jsonl` keeps growing in `report_dir`.



In [ ]:
# Start dashboard server — runs in a background thread, doesn't block training.

import os
import time as _time

try:
    from bnnr import start_dashboard
    dashboard_url = start_dashboard(
        run_root=config.report_dir,
        port=8080,
        auto_open=False,  # We handle display below
        build_frontend=False,  # Skip npm build on Colab
    )
    print(f"Dashboard server: {dashboard_url}")

    # On Google Colab: embed dashboard in notebook
    try:
        import google.colab  # noqa: F401
        from google.colab import output as colab_output

        _time.sleep(2)
        print("\n📊 Colab detected — embedding live dashboard below:")
        colab_output.serve_kernel_port_as_iframe(8080, height=900)

        # Optional: expose dashboard externally (phone/outside Colab session)
        # Requires: NGROK_AUTHTOKEN set in environment
        try:
            from pyngrok import ngrok

            token = os.environ.get("NGROK_AUTHTOKEN")
            if token:
                ngrok.set_auth_token(token)
                public_url = ngrok.connect(8080, "http").public_url
                print(f"🌍 Public dashboard URL: {public_url}")
            else:
                print("ℹ Optional public URL: install pyngrok and set NGROK_AUTHTOKEN.")
        except Exception as tunnel_exc:
            print(f"ℹ Tunnel not started: {tunnel_exc}")

    except ImportError:
        # Not on Colab — open URL manually
        print("Open the URL above in your browser to view the dashboard.")

except Exception as e:
    print(f"Dashboard not available: {e}")
    print("Training will continue without live monitoring.")
    print("Results are saved to the reports/ folder.")


## 8. Train!

BNNR will iteratively test each augmentation and select the best ones based on **F1 (samples)** — the primary multi-label metric.

The training flow is identical to single-label BNNR — the `task="multilabel"` config handles all the differences internally.

In [ ]:
trainer = BNNRTrainer(adapter, train_loader, val_loader, augmentations, config)
result = trainer.run()

## 9. Results

Let's inspect the training results — which augmentations did BNNR select, and how did F1 improve?

In [ ]:
print("=" * 50)
print("  RESULTS")
print("=" * 50)
print(f"  Best path         : {result.best_path}")
print(f"  Selected augs     : {result.selected_augmentations}")
print()
print("  Best metrics:")
for k, v in result.best_metrics.items():
    if k == "loss":
        print(f"    {k:<16s}: {v:.4f}")
    else:
        print(f"    {k:<16s}: {v:.4f}  ({v*100:.1f}%)")
print()
print(f"  Report: {result.report_json_path}")

### Replay events & plot metrics

If event logging was enabled, we can replay the full training history and plot metrics over time.

In [ ]:
from bnnr import replay_events

events_path = result.report_json_path.parent / "events.jsonl"
if events_path.exists():
    state = replay_events(events_path)
    mt = state.get("metrics_timeline", [])

    if mt:
        epochs = list(range(1, len(mt) + 1))
        f1s = [r.get("f1_samples", 0) * 100 for r in mt]
        f1m = [r.get("f1_macro", 0) * 100 for r in mt]
        acc = [r.get("accuracy", 0) * 100 for r in mt]
        loss = [r.get("loss", 0) for r in mt]

        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

        ax1.plot(epochs, f1s, "b-o", markersize=4, label="F1 Samples")
        ax1.plot(epochs, f1m, "g-s", markersize=4, label="F1 Macro")
        ax1.plot(epochs, acc, "r-^", markersize=4, label="Accuracy (subset)")
        ax1.set_xlabel("Epoch")
        ax1.set_ylabel("%")
        ax1.set_title("Multi-Label Metrics")
        ax1.legend()
        ax1.grid(True, alpha=0.3)

        ax2.plot(epochs, loss, "k-o", markersize=4)
        ax2.set_xlabel("Epoch")
        ax2.set_ylabel("Loss")
        ax2.set_title("BCE Loss")
        ax2.grid(True, alpha=0.3)

        plt.tight_layout()
        plt.show()
else:
    print("No events file found — set event_log_enabled=True in config.")

## 10. Summary — what changed vs single-label?

Moving from single-label to multi-label with BNNR requires **exactly two flags**:

```python
# Single-label (default)
adapter = SimpleTorchAdapter(model=model, criterion=nn.CrossEntropyLoss(), ...)
config = BNNRConfig(task="classification", ...)

# Multi-label — just add these two flags:
adapter = SimpleTorchAdapter(model=model, criterion=nn.BCEWithLogitsLoss(), multilabel=True, ...)
config = BNNRConfig(task="multilabel", ...)
```

Everything else — augmentations, XAI, ICD/AICD, pruning, dashboard, reporting — works identically. BNNR handles the internal differences (sigmoid vs argmax, F1 vs accuracy, per-label vs per-class analysis) automatically.

### For your own data

Replace the synthetic dataset with your real data:

```python
# Your multi-label dataset should return:
#   image: Tensor[C, H, W] with values in [0, 1]
#   label: Tensor[N_LABELS] multi-hot vector (0s and 1s)
#   index: int (sample index for XAI cache)
```

That's it! 🎉